In [ ]:
from azureml.core import Workspace, Dataset

datastore_name = '<insert_your_datastore_name>'
dataset_name = '<insert_fashion_mnist_dataset_name>'

workspace = Workspace.from_config()
dataset = Dataset.get_by_name(workspace, name=dataset_name)

In [ ]:
mount = dataset.mount()
mount.start()
mounted_input_path = mount.mount_point

In [ ]:
import os
os.listdir(mounted_input_path)

In [ ]:
import gzip

def convert(imgf, labelf, outf, n):
    f = gzip.open(imgf, "rb")
    temp = gzip.open(labelf, "rb")
    o = open(outf, "w")

    f.read(16)
    temp.read(8)
    images = []

    for i in range(n):
        image = [ord(temp.read(1))]
        for j in range(28 * 28):
            image.append(ord(f.read(1)))
        images.append(image)

    for image in images:
        o.write(",".join(str(pix) for pix in image) + "\n")
    f.close()
    o.close()
    temp.close()

In [ ]:
import os
mounted_output_path = '.'
convert(os.path.join(mounted_input_path, 'train-images-idx3-ubyte.gz'),
        os.path.join(mounted_input_path, 'train-labels-idx1-ubyte.gz'),
        os.path.join(mounted_output_path, 'mnist_train.csv'), 60000)

convert(os.path.join(mounted_input_path, 't10k-images-idx3-ubyte.gz'),
        os.path.join(mounted_input_path, 't10k-labels-idx1-ubyte.gz'),
        os.path.join(mounted_output_path, 'mnist_test.csv'), 10000)

In [ ]:
import pandas as pd

data_train = pd.read_csv('mnist_train.csv',header=None)
data_test = pd.read_csv('mnist_test.csv',header=None)


In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

X = np.array(data_train.iloc[:, 1:])
y = to_categorical(np.array(data_train.iloc[:, 0]))

# here we split validation data to optimiza classifier during training
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=13)

# test data
X_test = np.array(data_test.iloc[:, 1:])
y_test = to_categorical(np.array(data_test.iloc[:, 0]))


X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1).astype('float32') / 255
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1).astype('float32') / 255

batch_size = 256
num_classes = 10
epochs = 10

# construct neuron network
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_val, y_val))

score = model.evaluate(X_test, y_test, verbose=0)

# log a single value
print('Test loss:', score[0])

print('Test accuracy:', score[1])

plt.figure(figsize=(6, 3))
plt.title('Fashion MNIST with Keras ({} epochs)'.format(epochs), fontsize=14)
plt.plot(history.history['accuracy'], 'b-', label='Accuracy', lw=4, alpha=0.5)
plt.plot(history.history['loss'], 'r--', label='Loss', lw=4, alpha=0.5)
plt.legend(fontsize=12)
plt.grid(True)

# log an image

# create a ./outputs/model folder in the compute target
# files saved in the "./outputs" folder are automatically uploaded into run history
os.makedirs('./outputs/model', exist_ok=True)

# serialize NN architecture to JSON
model_json = model.to_json()
# save model JSON
with open('./outputs/model/model.json', 'w') as f:
    f.write(model_json)
# save model weights
model.save_weights('./outputs/model/model.h5')
print("model saved in ./outputs/model folder")

In [ ]:
# Register tabular version of fashion mnist dataset as TabularDaset in our datastore
from azureml.core import Datastore
datastore =  Datastore.get(workspace, datastore_name)
Dataset.Tabular.register_pandas_dataframe(data_train, datastore, "fashion_mnist_train_tabular", show_progress=True)
Dataset.Tabular.register_pandas_dataframe(data_test, datastore, "fashion_mnist_test_tabular", show_progress=True)